

```
# This is formatted as code
```

# Linear Classification

 Implement Linear Classification using pytorch. This consists of having fully connected layers connected one after the other and ReLu activation functions between them.

 Build a neural network with a minimun of 2 layers in order to do classification.

Permit the notebook to access your drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import time
import pdb
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline

torch.manual_seed(1)    # reproducible

Get the dataset

In [3]:
def get_train_valid_loader(data_dir='../data',
                           batch_size=64,
                           augment=False,
                           random_seed = 1,
                           valid_size=0.02,
                           shuffle=True,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False):
    """
    Utility function for loading and returning train and valid
    multi-process iterators over the CIFAR-10 dataset. A sample
    9x9 grid of the images can be optionally displayed.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )

    # visualize some images
    if show_sample:
        sample_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=9, shuffle=shuffle,
            num_workers=num_workers, pin_memory=pin_memory,
        )
        data_iter = iter(sample_loader)
        images, labels = data_iter.next()
        X = images.numpy().transpose([0, 2, 3, 1])
        plot_images(X, labels)

    return (train_loader, valid_loader)

trainloader, valloader = get_train_valid_loader()

100%|██████████| 170M/170M [00:18<00:00, 9.32MB/s]


Extracting ../data/cifar-10-python.tar.gz to ../data
Files already downloaded and verified


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Define the network

In [4]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()

        ################################################################################
        # TODO:                                                                        #
        # Define 2 or more different layers of the neural network                      #
        ################################################################################
        self.fc1 = torch.nn.Linear(n_feature, n_hidden)
        self.fc2 = torch.nn.Linear(n_hidden, n_output)
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################


    def forward(self, x):
        x = x.view(x.size(0),-1)
        ################################################################################
        # TODO:                                                                        #
        # Set up the forward pass that the input data will go through.                 #
        # A good activation function betweent the layers is a ReLu function.           #
        ################################################################################
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        return x

In [8]:
################################################################################
# TODO:                                                                        #
# Define the parameters of the network the way you want it to be.              #
# Choose an Optimizer that will be used to minimize the loss function.         #
################################################################################
net = Net(n_feature=3072, n_hidden=600, n_output=10)     # define the network
print(net)  # net architecture

# Loss and Optimizer (Try different learning rates)
# Softmax is internally computed.
# Set parameters to be updated.

optimizer = torch.optim.SGD(net.parameters(), lr=5e-4)  # Choose the optimizer you want and tune its hyperparameter
loss_func = torch.nn.CrossEntropyLoss()  # the target label is NOT an one-hotted
################################################################################
#                              END OF YOUR CODE                                #
################################################################################

Net(
  (fc1): Linear(in_features=3072, out_features=600, bias=True)
  (fc2): Linear(in_features=600, out_features=10, bias=True)
)


In [9]:
it = iter(trainloader)
X_batch, y_batch = next(it)
print(X_batch.shape)
print(y_batch.shape)

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [10]:
#traindataset = utils.TensorDataset(X_train, y_train)
#trainloader = utils.DataLoader(traindataset, batch_size=64, shuffle=True)

epochs = 10
steps = 0
print_every = 20
for e in range(epochs):
    start = time.time()
    for images, labels in iter(trainloader):
        steps += 1
        ################################################################################
        # TODO:                                                                        #
        # Run the training process                                                     #
        #                                                                              #
        ################################################################################
        optimizer.zero_grad()
        # no need to resize images, since it is done inside the forward() method.
        logits = net(images)  # net is an instance of torch.nn.Module, this will invoke __call__ and eventually
        # it will call the forward method on images.
        loss = loss_func(logits, labels)
        loss.backward()
        optimizer.step()
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################

        if steps % print_every == 0:
                stop = time.time()
                # Test accuracy
                net.eval()
                correct = 0
                total = 0
                with torch.no_grad():
                    for data in valloader:
                          images, labels = data
                          outputs = net(images)
                          _, predicted = torch.max(F.softmax(outputs).data, 1)
                          total += labels.size(0)
                          correct += (predicted == labels).sum().item()

                    print('Accuracy of the network on the %d val images: \
                    %d %%' % (total,100 * correct / total))

                start = time.time()

<ipython-input-10-4dfc50a9f1d5>:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  _, predicted = torch.max(F.softmax(outputs).data, 1)


Accuracy of the network on the 1000 val images:                     15 %
Accuracy of the network on the 1000 val images:                     16 %
Accuracy of the network on the 1000 val images:                     18 %
Accuracy of the network on the 1000 val images:                     21 %
Accuracy of the network on the 1000 val images:                     22 %
Accuracy of the network on the 1000 val images:                     23 %
Accuracy of the network on the 1000 val images:                     24 %
Accuracy of the network on the 1000 val images:                     24 %
Accuracy of the network on the 1000 val images:                     25 %
Accuracy of the network on the 1000 val images:                     25 %
Accuracy of the network on the 1000 val images:                     26 %
Accuracy of the network on the 1000 val images:                     26 %
Accuracy of the network on the 1000 val images:                     26 %
Accuracy of the network on the 1000 val images:    

After training, the model should be saved to be tested on the test dataset or to be used in a real-life application. To save your model in pytorch:

In [11]:
torch.save(net.state_dict(), 'drive/MyDrive/Colab Notebooks/linearClassifier_pytorch.ckpt')

Remeber the above path. You need to load your trained model in another notebook:

In [12]:
checkpoint = torch.load("drive/MyDrive/Colab Notebooks/linearClassifier_pytorch.ckpt")
net.load_state_dict(checkpoint)

<ipython-input-12-d44a4b42381a>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("drive/MyDrive/Colab Notebooks/linearClassifier_pytorch.ckpt")


<All keys matched successfully>